In [9]:
import os
os.path.join('../')
from collections import defaultdict, OrderedDict
from typing import Dict, List, Any, Callable

from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from doggos.fuzzy_sets import Type1FuzzySet
from doggos.fuzzy_sets.fuzzy_set import FuzzySet
from doggos.induction.information_system import InformationSystem
from doggos.inference import MamdaniInferenceSystem
from doggos.inference.defuzzification_algorithms import center_of_gravity, karnik_mendel
from doggos.inference.inference_system import InferenceSystem
from doggos.knowledge import Rule, Clause, fuzzify, LinguisticVariable, Domain
from doggos.knowledge.consequents import MamdaniConsequent
from doggos.knowledge.consequents.consequent import Consequent
from doggos.utils.grouping_functions import create_set_of_variables
from doggos.utils.membership_functions.membership_functions import generate_equal_gausses, sigmoid, gaussian

In [67]:
dataset_name = "Pima Indians Diabetes"
ds = pd.read_csv("../data/" + dataset_name + ".csv", sep=";")

ds.head()

,F0,F1,F2,F3,F4,F5,F6,F7,Decision
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,43642.0,0.351,31,0
2,8,183,64,0,0,43547.0,0.672,32,1
3,1,89,66,23,94,43493.0,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [69]:
pca = PCA(n_components=4)
values_no_decision = ds.drop(labels=["Decision"], axis=1)
min_max_scaler = MinMaxScaler()
values_no_decision = min_max_scaler.fit_transform(values_no_decision.values)
pca.fit(values_no_decision)
print(pca.explained_variance_ratio_)

[0.58037873 0.156019   0.08730092 0.0579577 ]


In [70]:
principal_components = pca.transform(values_no_decision)
cols = []
for idx in range(principal_components.shape[1]):
    cols.append(f'F{idx}')
pca_pd_ds = pd.DataFrame(principal_components, columns=cols)
pca_pd_ds['Decision'] = ds['Decision']
pca_pd_ds.head()

,F0,F1,F2,F3,Decision
0,-0.407224,0.280587,0.041664,0.017709,1
1,0.619142,-0.161374,0.005978,-0.148457,0
2,0.614697,0.256225,-0.016473,0.233501,1
3,0.620975,-0.267658,0.037189,-0.108237,0
4,-0.384516,-0.216077,0.076533,0.192233,1


In [71]:
pca_pd_ds.to_csv("../data/" + dataset_name + " PCA.csv", sep=";")
